## Projet EDA : NetFlix

En se basant sur un dataset repréentant les sorties Netflix (2008-2021) nous allons explorez les dfférentes métriques et répondre à différentes questions concernant les contenus NetFlix. <br>

Nous commencerons par une analyse exploratoire. <br>
Nous répondrons ensuite aux différentes questions listées ci-dessous. <br>

<span style="font-style: italic;"> N.B : La visualisation graphique sera effectué prncipalement avec les modules plotly et seaborn. Les graphiques dinamiques n'étant pas pris en compte par la plateforme GitHub, un instantané des graphiques sera présenté. Pour une visualisation en dynamique n'hésitez pas à télécharger le repository et le jouer en local. <br></span>

- Identifier les genres de contenus les plus présent sur la plateforme
- Identifier les pays producteurs de contenus
- Identifier quels pays produit le plus de contenus
- Identifier les genres les plus représentatifs par pays
- Identfier la présence de saisonalité dans les sorties de contenus

### Import des modules

In [558]:
import pandas as pd
import re 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go  
from plotly.subplots import make_subplots
from IPython.display import display, HTML
pd.set_option('display.max_columns', None)


### Import des données et EDA

In [559]:
# import des données
dataset = pd.read_csv("./CSV/netflix_titles.csv", encoding='iso-8859-1')
# -- On définit des variables pour une exploration rapide du dataset -- #
# cinq premières lignes du dataset
head_dataset = dataset.head(n=10)
# Statistiques de base du dataset
stats_basiques = dataset.describe()
# Trouver les valeurs manquantes
valeur_manquantes = dataset.isnull().sum()
# identifier le noms des colonnes
nom_des_colonnes = dataset.columns
# identifier le type des colonnes
shape_dataset = dataset.shape

In [560]:
head_dataset

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
5,s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, JosÃ© Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...


In [561]:
dataset['country'].isnull().sum()
print("Le dataset contient {} lignes et {} colonnes".format(dataset.shape[0], dataset.shape[1]))
print("Le dataset contient {} valeurs manquantes".format(dataset.isnull().sum().sum()))

Le dataset contient 8807 lignes et 12 colonnes
Le dataset contient 4307 valeurs manquantes


### Changement de la colonne date_added

Cette colonne contient les dates en format chaine de caractères, pour une meilleure manipulation des données on passe au format datetime.

In [562]:
dataset['date_added'] = pd.to_datetime(dataset['date_added'])

### Ajout d'une colonne "Is_international"

Nous allons crée une colonne pour determiner si un film à la caratéristique international.<br>
Un film international est un film dont la production est en majorité issue d'un autre pays que les au Etats-Unis.

In [563]:
def is_international(liste_genre):
    try:
        liste_genre = [genre.strip() for genre in liste_genre.split(',')]
        for genre in liste_genre:
            if re.match('^International Movies$', genre) or re.match('^International TV Shows$', genre):
                return 1
            else:
                return 0
    except:
        pass


# On applique la fonction à la colonne listed_in
dataset['is_international'] = dataset['listed_in'].apply(is_international)


In [564]:
""" 
La colonne listed_in contient plusieurs genres séprarés par des virgules.
Certains genre sont Movies et TV Show.
Cela donne un contenu de "type" Movie et de genre "Movie" ce qui n'est pas utile et pourrait fausser les résultats.
Une solution serait de supprimer les genres Movie et Tv Show de la colonne listed_in.
Avec la fonction apply, on peut appliquer une fonction à chaque ligne du dataset.
Cette fonction va retirer les genres Movie et Tv Show de la colonne listed_in.

En REGEX, Il est important de chercher les mots Movie/Tv Show entre des caractères spéciaux pour les cibler spécifiquement.

"""

def replace_genre(liste_genre):
    # je split la liste des genres par des virgules
    genres_a_supprimer = ['^TV Shows$', '^Movies$', "^International Movies$", "^International TV Shows$"]
    try:
        liste_genre = [genre.strip() for genre in liste_genre.split(',')]
        for genre in liste_genre:
            for genre_a_supprimer in genres_a_supprimer:
                if re.match(genre_a_supprimer, genre):
                    liste_genre.remove(genre)
        # je reconstitue la liste des genres en les séparant par des virgules
        liste_genre = ', '.join(liste_genre)
        return liste_genre
    except:
        pass


dataset['listed_in'] = dataset['listed_in'].apply(replace_genre)


    

### Identifier les genres dominant pour les films et les séries TV

In [565]:
dataset_movies = dataset[dataset['type'] == 'Movie']
dataset_tv_shows = dataset[dataset['type'] == 'TV Show']

In [566]:
top_genre_movies = dataset_movies['listed_in'].str.split(',').explode().str.strip().value_counts()
top_genre_tv_shows = dataset_tv_shows['listed_in'].str.split(',').explode().str.strip().value_counts()

html_output = f"<div style='display: flex;'><div style='margin-right: 20px;'>{top_genre_movies.to_frame().to_html()}</div><div>{top_genre_tv_shows.to_frame().to_html()}</div></div>"
display(HTML(html_output))


,listed_in
Dramas,2427
Comedies,1674
Documentaries,869
Action & Adventure,859
Independent Movies,756
Children & Family Movies,641
Romantic Movies,616
Thrillers,577
Music & Musicals,375
Horror Movies,357


In [567]:
fig = make_subplots(rows=2, 
                    cols=1, 
                    subplot_titles=['Nombre de films par genre', 'Nombre de Tv Shows par genre'],
                    shared_xaxes=False
                    )
fig.add_trace(go.Bar(x=top_genre_movies.index, y=top_genre_movies.values, name='Movie'), row=1, col=1)

fig.add_trace(go.Bar(x=top_genre_tv_shows.index, y=top_genre_tv_shows.values, name='Tv Show'), row=2, col=1)

fig.update_layout(height=750, width=800)
fig.show()

![image1](assets/01_contenus_par_genre.png)  

### Répartition des pays producteurs dans le monde

In [568]:
pays_producteurs = dataset['country'].str.split(',').explode().str.strip().unique()

pays_producteurs = pd.DataFrame(pays_producteurs, columns=['Pays'])

fig = px.choropleth(pays_producteurs, locations="Pays", color_discrete_sequence=["red"], locationmode="country names")
fig.update_layout(title_text='Pays producteur de contenu sur Netflix', title_x=0.5)
fig.show()


![image1](assets/02_repartition_pays.png)  

### Le volume de production par pays 


In [569]:
volume_production_pays = dataset['country'].str.split(',').explode().str.strip().value_counts()

volume_production_pays.head(n=10).to_frame()



,country
United States,3690
India,1046
United Kingdom,806
Canada,445
France,393
Japan,318
Spain,232
South Korea,231
Germany,226
Mexico,169


In [570]:
fig = go.Figure()
fig.add_trace(go.Bar(x=volume_production_pays.index, 
                     y=volume_production_pays.values, 
                     text=volume_production_pays.values, 
                     texttemplate='%{text:.2s}', 
                     textposition='outside')
                     )
fig.update_layout(title_text='Volume de production par pays', title_x=0.5)
fig.show()


data = {'Pays': volume_production_pays.index, 'Volume de production': volume_production_pays.values}
df = pd.DataFrame(data)

fig = px.choropleth(df, locations="Pays", color="Volume de production", color_continuous_scale=px.colors.sequential.OrRd,  range_color=[1, 500], locationmode="country names")
fig.update_layout(title_text='Production de contenu par pays', title_x=0.5)

fig.show()


![image1](assets/03_volume_production_pays.png)<br>

![image1](assets/04_MAP_volume_production_pays.png)<br>

### Identifier les genres dominant par pays 

In [571]:
# la liste des pays producteurs
liste_country = dataset['country'].dropna().str.split(',').explode().str.strip().unique()

def get_top_genre_movies(liste_country):
    liste_resultat = []
    for country in liste_country:
        dataset_country = dataset[dataset['country'].str.contains(country, na=False)]
        dataset_country_movies = dataset_country[dataset_country['type'] == 'Movie']
        top_genre_movies = dataset_country_movies['listed_in'].dropna().str.split(',').explode().str.strip().value_counts()
        for i in range(len(top_genre_movies)):
            liste_resultat.append({"Pays": country, "Genre": top_genre_movies.index[i], "Valeurs": top_genre_movies.values[i]})
    return liste_resultat

def get_top_genre_tvshow(liste_country):
    liste_resultat = []
    for country in liste_country:
        dataset_country = dataset[dataset['country'].str.contains(country, na=False)]
        dataset_country_tv_show = dataset_country[dataset_country['type'] == 'TV Show']
        top_genre_tv_shows = dataset_country_tv_show['listed_in'].dropna().str.split(',').explode().str.strip().value_counts()
        for i in range(len(top_genre_tv_shows)):
            liste_resultat.append({"Pays": country, "Genre": top_genre_tv_shows.index[i], "Valeurs": top_genre_tv_shows.values[i]})
    return liste_resultat

top_genre_country_movie = pd.DataFrame(get_top_genre_movies(liste_country))
top_genre_country_tvshow = pd.DataFrame(get_top_genre_tvshow(liste_country))

# Les résultats contient la somme par genre ce qui fausse le treemap et affiche des genres sans pays associés
# on supprime les lignes en question
for i in range(len(top_genre_country_movie)):
    if top_genre_country_movie['Pays'][i] == '':
        top_genre_country_movie.drop(i, inplace=True)

for i in range(len(top_genre_country_tvshow)):
    if top_genre_country_tvshow['Pays'][i] == '':
        top_genre_country_tvshow.drop(i, inplace=True)




In [572]:
labels_m = [genre for genre in top_genre_country_movie['Genre']]
parents_m = ['' for i in range(len(top_genre_country_movie))]
values_m = [valeur for valeur in top_genre_country_movie['Valeurs']]
pays_m = [p for p in top_genre_country_movie['Pays']]

# je créer un DataFrame avec les listes pays, labels, values et parents
data_m = {'Pays': pays_m, 'labels': labels_m, 'values': values_m, 'parents': parents_m}
df_m = pd.DataFrame(data_m)

fig = px.treemap(df_m, path=['Pays', 'labels'], values='values', color='values', color_continuous_scale='RdBu')
fig.update_layout(title_text="Top Movie genre par pays", title_x=0.5)
fig.show()


labels_m = [genre for genre in top_genre_country_tvshow['Genre']]
parents_m = ['' for i in range(len(top_genre_country_tvshow))]
values_m = [valeur for valeur in top_genre_country_tvshow['Valeurs']]
pays_m = [p for p in top_genre_country_tvshow['Pays']]

# je créer un DataFrame avec les listes pays, labels, values et parents
data_m = {'Pays': pays_m, 'labels': labels_m, 'values': values_m, 'parents': parents_m}
df_m = pd.DataFrame(data_m)

fig = px.treemap(df_m, path=['Pays', 'labels'], values='values', color='values', color_continuous_scale='RdBu')
fig.update_layout(title_text="Top TV Show genre par pays", title_x=0.5)
fig.show()

/usr/lib/python3/dist-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/lib/python3/dist-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



/usr/lib/python3/dist-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/lib/python3/dist-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



![image1](assets/05_Top_genre_movie.png)<br>

![image1](assets/06_Top_genre_TV_show.png)<br>

### Identifier s'il y a une saisonalité dans les sorties de contenues sur la plateforme

In [573]:
#je selectionne les années de 2017 à 2021	
liste_years = [2016, 2017, 2018,2019,2020,2021]
liste_month = ["January","February","March","April","May","June","July","August","September","October","November","December"]
liste_release_per_month = []

for year in liste_years:
    dataset_year = dataset[dataset['date_added'].dt.year == year]
    result = dataset_year['date_added'].dt.month.value_counts().sort_index()
    liste_release_per_month.append({"Year": year, "Release by Month": result})

fig = make_subplots(rows=3, 
                    cols=2, 
                    shared_xaxes=False 
                    )

for i in range(1, len(liste_release_per_month)+1):
    if i <= 3:
        fig.add_trace(go.Bar(
                x=liste_month,
                y=liste_release_per_month[i-1]['Release by Month'].values, 
                text=liste_release_per_month[i-1]['Release by Month'].values, 
                texttemplate='%{text:.2s}', 
                textposition='outside', 
                name=liste_release_per_month[i-1]['Year']), 
                row=i, 
                col=1)
    else:
        fig.add_trace(go.Bar(
                x=liste_month,
                y=liste_release_per_month[i-1]['Release by Month'].values, 
                text=liste_release_per_month[i-1]['Release by Month'].values, 
                texttemplate='%{text:.2s}', 
                textposition='outside', 
                name=liste_release_per_month[i-1]['Year']), 
                row=i-3, 
                col=2)
fig.update_layout(height=1000, width=900)
fig.update_layout(title_text='Sorties Netflix par mois (2016-2021)', title_x=0.5)
fig.show()


![image1](assets/07_saisonalité.png)<br>

En se basant sur les sorties Netflix par mois entre 2016 et 2021 on constate qu'aucune tendance ne se dessine. <br>
Une autre approche pourrait permettre de toruver une tendance. 


In [574]:
calendirer_sortie = dataset.groupby('date_added')['show_id'].count()

fig = go.Figure(
    data = go.Scatter(
        x = calendirer_sortie.index, 
        y = calendirer_sortie.values,
       ),
    layout = go.Layout(
        title = go.layout.Title(text = "Sorties Netflix (2008-2021)", x = 0.5),
        xaxis = go.layout.XAxis(title = 'Date à laquelle les sorties ont été effectuées', rangeslider = go.layout.xaxis.Rangeslider(visible = True)),
        yaxis = go.layout.YAxis(title = 'Volumz de sortie')
    )
)

fig.show(renderer="notebook_connected")

![image1](assets/08_saisonalité-2.png)<br>

Sur le graphique ci-dessus, nous avons representé l'ensemble de sosrites sur la période 2008-2021.<br>
La répartition journalière nous permet d'observer une répétition à partr de 2016, chaque pic de sortie correspond au premier jour du mois. <br>
**On peut en conclure que la plateforme Netflix concentre ses sorties au premier jour du mois.** 